In [ ]:
# This code is contributed by Susobhan Akhuli. https://www.geeksforgeeks.org/introduction-to-levenshtein-distance/

# Python program for the above approach
def levenshtein_two_matrix_rows(str1, str2):
    # Get the lengths of the input strings
    m = len(str1)
    n = len(str2)

    # Initialize two rows for dynamic programming
    prev_row = [{
            "distance": j,
            "operation": ("Start"),
            "history": dict()
        } for j in range(n + 1)]
    curr_row = [0] * (n + 1)

    # Dynamic programming to fill the matrix
    for i in range(1, m + 1):
        # Initialize the first element of the current row
        curr_row[0] = {
            "distance": i,
            "operation": ("Start"),
            "history": dict()
        }

        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                # Characters match, no operation needed
                #curr_row[j] = prev_row[j - 1] #History doesn't matter
                curr_row[j] = { #History does matter
                    "distance": prev_row[j - 1]["distance"],
                    "operation": ("Nothing", str1[i - 1]),
                    "history": prev_row[j - 1]
                }
            else:
                # Choose the minimum cost operation
                curr_row[j] = min([
                    {
                        "distance": 1 + prev_row[j - 1]["distance"],    # Replace
                        "operation": ("Substitute", str1[i - 1], str2[j - 1]),
                        "history": prev_row[j- 1]
                    },
                    {
                        "distance": 1 + curr_row[j - 1]["distance"],  # Insert
                        "operation": ("Insert", str2[j - 1]),
                        "history": curr_row[j - 1]
                    },{
                        "distance": 1 + prev_row[j]["distance"],      # Remove
                        "operation": ("Remove", str1[i - 1]),
                        "history": prev_row[j]
                    }
                ], key=lambda cell: cell["distance"])

        # Update the previous row with the current row
        prev_row = curr_row.copy()

    # The final element in the last row contains the Levenshtein distance
    return curr_row[n]

# Example input strings
str1 = "kitten"
str2 = "sitting"

# Function call to calculate Levenshtein distance
distance = levenshtein_two_matrix_rows(str1, str2)

# Print the result
print("Levenshtein Distance:", distance)
import json
print("Json:", json.dumps(distance))

data = distance

In [ ]:
import graphviz

# Create a graphviz Digraph object
dot = graphviz.Digraph()

# Function to add nodes and edges to the graph in reverse order, skipping nodes with operation 'Nothing'
def add_nodes_edges_reverse_skip_nothing(data, parent=None):
    if data['operation'][0] == 'Nothing':
        if data['history']:
            add_nodes_edges_reverse_skip_nothing(data['history'], parent)
        return
    
    node_id = str(id(data))
    label = f"Distance: {data['distance']}\nOperation: {data['operation']}"
    dot.node(node_id, label)
    
    if parent:
        dot.edge(node_id, parent)
    
    if data['history']:
        add_nodes_edges_reverse_skip_nothing(data['history'], node_id)

# Add nodes and edges starting from the root
add_nodes_edges_reverse_skip_nothing(data)

# Save the graph to a file
dot.save('graph_reverse_skip_nothing_new.dot')

# Render the graph to a PNG image file
dot.render('graph_reverse_skip_nothing_new', format='png')

print("Graph has been successfully created and saved as graph_reverse_skip_nothing_new.png.")